In [ ]:
import os
import sys
from pathlib import Path

from rich import print

In [3]:
path = Path(os.getcwd()).parent# / "text_processing"
sys.path.append(str(path))

In [4]:
from text_processing.chunkers import SemanticChunker
from text_processing.doc_models.documents import Document


In [5]:
chunker = SemanticChunker()
chunker

In [6]:
test_files = list(Path.cwd().parent.glob('data/sample_pdf/**/*.pdf'))
test_files

[PosixPath('/home/thankgod/2025/tiiqu/textract-tiquu/data/sample_pdf/JSR_1778.pdf'),
 PosixPath('/home/thankgod/2025/tiiqu/textract-tiquu/data/sample_pdf/JSR_1774.pdf'),
 PosixPath('/home/thankgod/2025/tiiqu/textract-tiquu/data/sample_pdf/What_is_Sustainability-1.pdf'),
 PosixPath('/home/thankgod/2025/tiiqu/textract-tiquu/data/sample_pdf/JSR_1775.pdf')]

In [7]:
from marker.config.parser import ConfigParser
from marker.converters.pdf import PdfConverter
from marker.models import create_model_dict

config = {
    "output_format": "markdown",
    "ADDITIONAL_KEY": "VALUE"
}
config_parser = ConfigParser(config)

converter = PdfConverter(
    config=config_parser.generate_config_dict(),
    artifact_dict=create_model_dict(),
    processor_list=config_parser.get_processors(),
    renderer=config_parser.get_renderer(),
    llm_service=config_parser.get_llm_service()
)

rendered_file = converter(filepath=str(test_files[0]))

Recognizing Text: 100%|██████████| 1/1 [00:00<00:00,  2.02it/s]
Detecting bboxes: 0it [00:00, ?it/s]
Recognizing tables: 100%|██████████| 1/1 [00:00<00:00,  1.96it/s]


In [8]:
rendered_file

MarkdownOutput(markdown='### *Article*\n\n# **Urban Development and Land Use/Land Cover Changes in the European Borderlands of Nice and Monaco**\n\n### **Arian Behradfar 1,2, Rui Alexandre Castanho 2,3,\\*, Paulo Ferreira <sup>2</sup>**\n\n- <sup>1</sup> Department of Geomatics and Spatial Information Engineering, College of Engineering, University of Tehran, Tehran 14174-66191, Iran; arianbehradfar@ut.ac.ir (AB)\n- <sup>2</sup> VALORIZA—Research Centre for Endogenous Resource Valorization, Portalegre Polytechnic University, Portalegre 7300-110, Portugal; pferreira@ipportalgre.pt (PF)\n- <sup>3</sup> Advanced Research Centre, European University of Lefke, Lefke 99010, Northern Cyprus, TR-10 Mersin, Turkey\n- **\\*** Correspondence: Rui Alexandre Castanho, Email: acastanho@wsb.edu.pl, rui.castanho@ipportalegre.pt, Tel.: +351-912-494-673\n\n# **ABSTRACT**\n\nUrban development in European cross-border regions significantly influences LULC changes, affecting environmental sustainability an

In [9]:
sample_text = rendered_file.markdown
#print(sample_text)

In [10]:
# Create document and process it
document = Document(sample_text, doc_id="sample_doc")

vars(document)

{'text': '### *Article*\n\n# **Urban Development and Land Use/Land Cover Changes in the European Borderlands of Nice and Monaco**\n\n### **Arian Behradfar 1,2, Rui Alexandre Castanho 2,3,\\*, Paulo Ferreira <sup>2</sup>**\n\n- <sup>1</sup> Department of Geomatics and Spatial Information Engineering, College of Engineering, University of Tehran, Tehran 14174-66191, Iran; arianbehradfar@ut.ac.ir (AB)\n- <sup>2</sup> VALORIZA—Research Centre for Endogenous Resource Valorization, Portalegre Polytechnic University, Portalegre 7300-110, Portugal; pferreira@ipportalgre.pt (PF)\n- <sup>3</sup> Advanced Research Centre, European University of Lefke, Lefke 99010, Northern Cyprus, TR-10 Mersin, Turkey\n- **\\*** Correspondence: Rui Alexandre Castanho, Email: acastanho@wsb.edu.pl, rui.castanho@ipportalegre.pt, Tel.: +351-912-494-673\n\n# **ABSTRACT**\n\nUrban development in European cross-border regions significantly influences LULC changes, affecting environmental sustainability and regional plan

In [11]:
document

Document(doc_id=sample_doc, word_count=8883)

In [12]:
all_chunks = document.get_all_chunks()
print(f"Total chunks across all blocks: {len(all_chunks)}")
all_chunks[0].text

Total chunks across all blocks: 0

IndexError: list index out of range

In [13]:
# Step 1: Split into blocks
blocks = document.create_blocks(min_words=150)  # Lower threshold for demo

blocks

[Block(block_id=0, word_count=218),
 Block(block_id=1, word_count=850),
 Block(block_id=2, word_count=314),
 Block(block_id=3, word_count=705),
 Block(block_id=4, word_count=457),
 Block(block_id=5, word_count=2491),
 Block(block_id=6, word_count=1178),
 Block(block_id=7, word_count=660)]

In [14]:
print(f"Document split into {len(blocks)} blocks:")
for block in blocks:
    print(f"  {block}")

print("\n" + "="*50 + "\n")

Document split into 8 blocks:

Block(0): 218 words - Urban development in European cross-border regions...

Block(1): 850 words - Changes in LULC significantly impact global enviro...

Block(2): 314 words - The territory of Monaco, which comprises a dense c...

Block(3): 705 words - CLC has been selected as the dataset for LULC asse...

Block(4): 457 words - The spatial distribution of urban development and ...

Block(5): 2491 words - At the first step, we present the resulting LULC m...

Block(6): 1178 words - Regional planning in the Nice–Monaco area has emph...

Block(7): 660 words - In this study, we designed and employed a hybrid a...

==================================================

In [17]:




# Step 2: Split each block into chunks
for block in blocks:
    chunks = block.create_chunks(chunker, similarity_threshold=0.45)
    
    print(f"Block {block.block_id} ('{block.start_header}') split into {len(chunks)} chunks:")
    for chunk in chunks:
        print(f"  Chunk {chunk.chunk_id}: {chunk.word_count} words")
        print(f"    {chunk.preview(100)}")
    print()

# Example of getting all chunks from the document
all_chunks = document.get_all_chunks()
print(f"Total chunks across all blocks: {len(all_chunks)}")

                                                            

Block 0 ('**ABSTRACT**') split into 2 chunks:

Chunk 0: 134 words

Urban development in European cross-border regions significantly influences LULC changes, affecting ...

Chunk 1: 85 words

Wetlands and water bodies remained relatively stable during this period Rapid urbanization in the Ni...

...

Chunk 3: 127 words

Additionally, fostering cooperation

in policy-making and resource management can lead to more effec...

Chunk 4: 66 words

It has a historical commitment to preserving these marine ecosystems, emphasizing the importance of ...

Total chunks across all blocks: 52

In [16]:
print(all_chunks[4].text)

IndexError: list index out of range